In [17]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import pickle
import mgzip
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pyreadr

tqdm.pandas()

### Choose file we want to predict

In [12]:
with mgzip.open('../data/predict_bert_topic.mgzip', 'rb') as handle:
    predict = pickle.load(handle)
#predict.to_csv('../data/predict_bert_topic.csv')

In [13]:
X = predict
X = X.dropna()
X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
X.drop('newspaper', inplace=True, axis=1)

/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['tagespiegel', 'welt', 'zeit']] = pd.get_dummies(X['newspaper'])
/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/3869162192.py:3: SettingWithCopyWarning: 
A value is tr

#### Continuous labels

In [14]:
y = X['bert_score']
X.drop(['bert_score', 'bert_label'], axis=1, inplace=True)

/var/folders/06/ng7ccz8d4fdfcgp2wp0g434w0000gn/T/ipykernel_69626/2893972162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(['bert_score', 'bert_label'], axis=1, inplace=True)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

##### Create baseline value using linear regression

In [19]:
regr = LinearRegression()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.06330454354398718

In [20]:
y_pred = regr.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 0.17404903048763853


R2 is 6%, MAE is 0.174

##### Try out ensemble random forest regressor
Grid search for parameter tuning, cross validation: 5

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5 , cv = 3, verbose = 3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

rf_random.best_params_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


In [ ]:
regr = RandomForestRegressor(verbose=1, n_jobs=-1)
#regr = RandomForestRegressor(verbose = 1, n_jobs = -1, n_estimators = 600, min_samples_split = 10, min_samples_leaf = 4, max_features = 'sqrt', max_depth = 90, bootstrap = False)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
y_pred = regr.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))